General Overview:
1. Understand the shape of the data
2. Data Cleaning
3. Data Exploration
4. Feature Enginerring
5. Data Preprocessing for Model
6. Basic Model Buiilding
7. Model Tuning
8. Ensemble Model Building
9. Results

### 1. Understand the shape of the data

In [ ]:
import numpy as np
import pandas as pd
import re
import calendar

import seaborn as sns
import matplotlib.style as style


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline